In [1]:
import os.path
import sys
sys.path.insert(0, '../../')
from src.questions_construction.questions import FLUENT_TYPES_LIST
from src.questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from src.questions_construction.domains import DOMAIN_NAMES
from src.analysis.model_performances import * #gather_questions, TRANSPORTATION_DOMAINS, NON_TRANSPORTATION_DOMAINS
from src.common import *
import random
from collections import defaultdict
from copy import deepcopy
import itertools
from src.analysis.model_performances import *

In [2]:
questions_dir = f'{DATA_PATH}/questions_m1'
ids_file_name = 'dataset_ids.test'
selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl')
questions_by_id = gather_questions(questions_dir, selected_ids)

In [3]:
data_columns = defaultdict(int)
for d in questions_by_id.values():
    data_columns[(d['question_category'], d['answer_type'])]+=1
data_columns

defaultdict(int,
            {('object_tracking', 'true_false_answer'): 1598,
             ('object_tracking', 'free_answer'): 120,
             ('fluent_tracking', 'true_false_answer'): 1889,
             ('fluent_tracking', 'free_answer'): 952,
             ('state_tracking', 'true_false_answer'): 153,
             ('state_tracking', 'free_answer'): 101,
             ('action_executability', 'free_answer'): 119,
             ('action_executability', 'true_false_answer'): 240,
             ('effects', 'free_answer'): 108,
             ('effects', 'true_false_answer'): 957,
             ('numerical_reasoning', 'true_false_answer'): 240,
             ('numerical_reasoning', 'free_answer'): 120,
             ('hallucination', 'true_false_answer'): 240,
             ('hallucination', 'free_answer'): 240})

In [ ]:
def to_df(results_all, model_name, answer_type, prompt_type, subs, domains):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
            data_columns = {}
            for domain in domains:
                by_fluent_type = {}
                for fluent_type in list(FLUENT_TYPES_LIST) + [None]:
                    data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, substitutions, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
                    stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, substitutions)
                    by_fluent_type[fluent_type] = stats.compute()['result']
                    
                
                data_columns[domain] = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
            data.append(data_columns)
    return pd.DataFrame(data, index = index)